# Computação Gráfica e Realidade Aumentada 
## &copy; 2025
<author>Authors: Guilherme Cabaço e Henrique Gomes</author>
<h1>Assignment 1</h1>

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

### Vertex e Fragment Shader

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void) {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

# A.1
### Cube class

In [ ]:
%%html

<script id="cube">
class cube extends CGRAobject{
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        
        var vertices = [
          -0.5,-0.5,-0.5,  //0
          -0.5,-0.5, 0.5,  //1
          -0.5, 0.5,-0.5,  //2
          -0.5, 0.5, 0.5,  //3
           0.5,-0.5,-0.5,  //4
           0.5,-0.5, 0.5,  //5
           0.5, 0.5,-0.5,  //6
           0.5, 0.5, 0.5   //7
        ];

        var indices = [
          1,5,7, 1,7,3,  // front
          0,2,6, 0,6,4,  // back
          0,1,3, 0,3,2,  // left
          4,6,7, 4,7,5,  // right
          2,3,7, 2,7,6,  // top
          0,4,5, 0,5,1   // bottom
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex
            
          //If x==-0.5 assign color1 otherwise its the other side and assign color2  
          const t = (vertices[i] + 0.5);
            
          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }
        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = this.modelMat;
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);

        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>

### Sphere

In [ ]:
%%html

<script id="sphere">

class sphere extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;

        var nCircles = 12;  // number of circles
        var nVertices = 12;  // number of vertices per circle
        var radius = 0.5;

        var vertices = [];
        var indices = [];
        var colors = [];

        // Top vertice
        vertices.push(0.0, radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const topIndex = 0;

        // Generate middle circles
        for (let i = 1; i < nCircles; i++) {
            
            var r = radius * Math.sin(i / nCircles * Math.PI);
            
            var y = radius * Math.cos(i / nCircles * Math.PI);
            
            for (let j = 0; j < nVertices; j++) {
                
                var x = r * Math.cos(j / nVertices * 2 * Math.PI);
                var z = r * Math.sin(j / nVertices * 2 * Math.PI);
                
                vertices.push(x, y, z);
                
                var t = j/nVertices; // based on x coordinate
                colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
                );
            }
        }

        // Bottom vertice
        vertices.push(0.0, -radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const bottomIndex = vertices.length / 3 - 1;

        // Indices
        // Top 
        for (let j = 1; j < nVertices; j++) {
            const next = (j % nVertices) +1;
            indices.push(topIndex, j, next);
        }

        // Middle stacks
        for (let i = 0; i < nCircles - 2; i++) {
            var start = 1 + i * nVertices;
            var next = start + nVertices;
            
            for (let j = 0; j < nVertices; j++) {
                const k = (j % nVertices) + 1;
            
                indices.push(start + j, next + j, next + k);
                indices.push(start + j, next + k, start + k);
            }
        }

        // Bottom 
        var lastRingStart = 1 + (nCircles - 2) * nVertices;
        for (let j = 0; j < nVertices; j++) {
            const next = (j % nVertices) + 1;
            indices.push(lastRingStart + j, bottomIndex, lastRingStart + next);
        }
        

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const MVP = projMat['*'](viewMat['*'](this.modelMat));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}


</script>

### Cylinder

In [ ]:
%%html

<script id="cylinder">
class cylinder extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);

        const n = 12;
        const radius = 0.5;
        const height = 1.0;

        const vertices = [];
        const colors = [];
        const indices = [];

        //Top and base circle vertices
        for (let ring = 0; ring < 2; ring++) {
            var y = ring === 0 ? -height / 2 : height / 2;
            for (let i = 0; i < n; i++) {
                
                var x = radius * Math.cos((i / n) * 2 * Math.PI);
                var z = radius * Math.sin((i / n) * 2 * Math.PI);
                vertices.push(x, y, z);

                var t = i/n; // Gradient along the vertices
                colors.push(
                    color1[0] * (1 - t) + color2[0] * t,
                    color1[1] * (1 - t) + color2[1] * t,
                    color1[2] * (1 - t) + color2[2] * t
                );
            }
        }

        // Top and base center vertices
        var baseIndex = vertices.length / 3;
        vertices.push(0, -height / 2, 0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        var topIndex = vertices.length / 3;
        vertices.push(0, height / 2, 0);
         colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );


        //Side indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n;
            var bottom1 = i;
            var bottom2 = next;
            var top1 = i + n;
            var top2 = next + n;

            // First triangle
            indices.push(bottom1, top1, bottom2);
            // Second triangle
            indices.push(bottom2, top1, top2);
        }

        // Top indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 1 again
            indices.push(topIndex, i+n, next+n);
        }
        
        // Base indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 1 again
            indices.push(baseIndex, next, i);
        }


        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = this.modelMat;
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


### Cone

In [ ]:
%%html

<script id="cone">

class cone extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;
        
        var n = 12;
        var radius = 0.5;
        var height = 1.0;
        var vertices = [];
        var indices = [];
        var colors = [];
        
        // top center vertice
        vertices.push(0.0, height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        
        // base vertices
        for (let i = 0; i < n; i++) {
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = -height/2;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);
            
          var t = i/n; //Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }
        
        // base center vertice
        vertices.push(0.0, -height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        
        var topIndex = 0;
        var baseIndex = n + 1;
        
        // Indices
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(topIndex, i, next);
        }
        
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(baseIndex, next, i);
        }

        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = this.modelMat;
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);

        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }

}

</script>

### Disk

In [ ]:
%%html

<script id="disk">
class disk extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1], n = 12) {
        super(glcontext);

        var radius = 0.5;

        var vertices = [0.0, 0.0, 0.0]; //Center vertex
        
        var colors = [];
        // Center color 
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        for (var i = 0; i < n; i++) {
            
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = 0.0;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }

        var indices = [];
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(0, i, next);
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = this.modelMat;
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


### Pyramid

In [ ]:
%%html

<script id="pyramid">
class pyramid extends CGRAobject {
      constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);


        var vertices = [

          -0.5, -0.5, -0.5, // 0
           0.5, -0.5, -0.5, // 1
           0.5, -0.5,  0.5, // 2
          -0.5, -0.5,  0.5, // 3

           0.0,  0.5,  0.0  // 4
        ];

        var indices = [
          // sides
          0, 1, 4, //Left side
          1, 2, 4,
          2, 3, 4,
          3, 0, 4,

          // base
          0, 1, 2,
          0, 2, 3
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex

          //If x==-0.5 assign color1 otherwise assign color2  
          const t = (vertices[i] + 0.5);

          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
  }
    
  draw(viewMat, projMat, shader) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = this.modelMat;
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


# Main

In [ ]:
%%html
<canvas id="myCanvas2" width="800" height="800" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myapp2 extends DEECapp{
    
    orientation = 0; // orientation of the camera
    mode = 0; // Visitor == 0 or Train == 1
    shape = 0; //Shape for the train wheels
    
    cameraPos = glm.vec3(0, 1, -30); //Initial position of the camera
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        //===========================================================
        // Initialize objects
        
        //Pyramids
        this.bigpyramid1 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
        this.bigpyramid1.setShader(this.shaderprog);
        
        this.bigpyramid2 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
        this.bigpyramid2.setShader(this.shaderprog);
        
        this.bigpyramid3 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
        this.bigpyramid3.setShader(this.shaderprog);
        
        
        //Sun
        
        this.sun = new sphere(this.gl, [0.970,0.558,0.0194], [0.970,0.558,0.0194]);
        this.sun.setShader(this.shaderprog);
        
        
        //Random Objects
        this.cone = new cone(this.gl);
        this.cone.setShader(this.shaderprog);
        
        this.disk = new disk(this.gl);
        this.disk.setShader(this.shaderprog);
        
        this.cube = new cube(this.gl);
        this.cube.setShader(this.shaderprog);
        
        this.cylinder = new cylinder(this.gl);
        this.cylinder.setShader(this.shaderprog);
        
        this.sphere = new sphere(this.gl);
        this.sphere.setShader(this.shaderprog);
        
        //Floor and track
        this.floor = new disk(this.gl, [0.950,0.816,0.333], [0.950,0.816,0.333]);
        this.floor.setShader(this.shaderprog);
        
        this.inner_floor = new disk(this.gl, [0.950,0.816,0.333], [0.950,0.816,0.333], 60);
        this.inner_floor.setShader(this.shaderprog);
        
        this.track = new disk(this.gl, [0.6,0.6,0.6], [0.6,0.6,0.6], 60);
        this.track.setShader(this.shaderprog);
        //===========================================================
        
        //===========================================================
        //Translations
        this.trans_bigpyramid1 = glm.translate(glm.mat4(1.0),glm.vec3(700,37,650));
        this.trans_bigpyramid2 = glm.translate(glm.mat4(1.0),glm.vec3(500,37,550));
        this.trans_bigpyramid3 = glm.translate(glm.mat4(1.0),glm.vec3(300,17,450));
        
        this.trans_sun = glm.translate(glm.mat4(1.0),glm.vec3(400,200,700));
        
        //Random Objects
        this.trans_cone = glm.translate(glm.mat4(1.0),glm.vec3(-5,0.5,5));
        this.trans_disk = glm.translate(glm.mat4(1.0),glm.vec3(7,0.2,-3));
        this.trans_cube = glm.translate(glm.mat4(1.0),glm.vec3(-1,0.5,-10));
        this.trans_cylinder = glm.translate(glm.mat4(1.0),glm.vec3(3,0.5,-7));
        this.trans_sphere = glm.translate(glm.mat4(1.0),glm.vec3(0,0.5,0));
        
        //Floor and track
        this.trans_track = glm.translate(glm.mat4(1.0),glm.vec3(0,0.01,0));
        this.trans_inner_floor = glm.translate(glm.mat4(1.0),glm.vec3(0,0.02,0));
        
        //===========================================================
        
        //===========================================================
        //Scales
        this.floor_scale = glm.scale(glm.mat4(1.0),glm.vec3(1000,0,1000));
        this.inner_floor_scale = glm.scale(glm.mat4(1.0),glm.vec3(65,0,30));
        
        this.track_scale = glm.scale(glm.mat4(1.0),glm.vec3(70,0,35));
        
        this.sun_scale = glm.scale(glm.mat4(1.0),glm.vec3(50,50,50));
        
        this.bigpyramid1_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
        this.bigpyramid2_scale = glm.scale(glm.mat4(1.0),glm.vec3(150,115,150));
        this.bigpyramid3_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
        //===========================================================
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
  
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    keyprocess(evt){
        var step=1;
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 118: // v
                 if(this.mode == 0){
                     this.mode = 1; // Passenger
                     this.viewM = glm.lookAt(glm.vec3(0,0,1),glm.vec3(5,0,10),glm.vec3(0,1,0));
                 }
                 else if(this.mode == 1){
                     this.mode = 0; //Visitor
                     this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
                 }
                 break;
             
             
             case 119: // w
                 if (this.mode == 0){
                     this.cameraPos.x += step*Math.sin(glm.radians(this.orientation));
                     this.cameraPos.z += step*Math.cos(glm.radians(this.orientation));
                     
                     this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
                 }
                 break;
             
             case 115: //s
                 if (this.mode == 0){
                     this.cameraPos.x -= step*Math.sin(glm.radians(this.orientation));
                     this.cameraPos.z -= step*Math.cos(glm.radians(this.orientation));
                     
                     this.viewM = glm.lookAt(this.cameraPos, glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
                 }
                 break;
             
             case 97: //
                 if (this.mode == 0){
                     this.orientation+=2;
                     this.viewM = glm.lookAt(this.cameraPos, glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
                 }
                 break;
             
             case 100: //d
                 if (this.mode == 0){
                     this.orientation-=2;
                     this.viewM = glm.lookAt(this.cameraPos, glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))), glm.vec3(0,1,0));
                 }
                 break;
             
             case 32: //switch shape
                 if (this.shape == 0){
                     this.shape = 1;
                     this.bigpyramid1 = new sphere(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
                     this.bigpyramid1.setShader(this.shaderprog);
                     this.bigpyramid1_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,100,100));
                 }
                 else if (this.shape == 1){
                     this.shape = 2;
                     this.bigpyramid1 = new cube(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
                     this.bigpyramid1.setShader(this.shaderprog);
                     this.bigpyramid1_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,100,100));
                     
                 }
                 else if (this.shape == 2){
                     this.shape = 0;
                     this.bigpyramid1 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
                     this.bigpyramid1.setShader(this.shaderprog);
                     this.bigpyramid1_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
                 }
                 break;
         }
    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT);
        
        //Background color
        this.gl.clearColor(0, 0, 1, 0.2);
        
        //===========================================================
        //Track and floor
        this.floor.setModelTransformation(this.floor_scale);
        this.floor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.inner_floor.setModelTransformation(this.trans_inner_floor['*'](this.inner_floor_scale));
        this.inner_floor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.track.setModelTransformation(this.trans_track['*'](this.track_scale));
        this.track.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Pyramids
        this.bigpyramid1.setModelTransformation(this.trans_bigpyramid1['*'](this.bigpyramid1_scale));
        this.bigpyramid1.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.bigpyramid2.setModelTransformation(this.trans_bigpyramid2['*'](this.bigpyramid2_scale));
        this.bigpyramid2.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.bigpyramid3.setModelTransformation(this.trans_bigpyramid3['*'](this.bigpyramid3_scale));
        this.bigpyramid3.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Sun 
        this.sun.setModelTransformation(this.trans_sun['*'](this.sun_scale));
        this.sun.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Random Objects
        this.cone.setModelTransformation(this.trans_cone);
        this.cone.draw(this.viewM, this.projectionM, this.shaderprog);
           
        this.disk.setModelTransformation(this.trans_disk);
        this.disk.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.cube.setModelTransformation(this.trans_cube);
        this.cube.draw(this.viewM, this.projectionM, this.shaderprog);

        this.cylinder.setModelTransformation(this.trans_cylinder);
        this.cylinder.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.sphere.setModelTransformation(this.trans_sphere);
        this.sphere.draw(this.viewM, this.projectionM, this.shaderprog);
        
        //===========================================================

    }
    
}


var app2 = new myapp2('myCanvas2');

app2.run();

</script>